In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from statistics import mean
from collections import Counter
from itertools import groupby
import numpy
import matplotlib.pyplot as plt
import math
import numpy as np

from statsmodels.tsa.ar_model import AutoReg

from statsmodels.tsa.arima.model import ARIMA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r purchase_history.csv drive/MyDrive/

In [2]:
df = pd.read_csv("purchase_history.csv")
df.head()

,user_id,product_id,created_at
0,647746,2011564,2019-02-09
1,906136,4105782,2019-11-23
2,12862997,2144063,2019-07-16
3,12097079,1207738,2019-04-05
4,3184486,4278839,2019-03-08


In [12]:
df_user = df.groupby(['user_id'], as_index=False).agg({"created_at" : lambda x: list(sorted(set(x)))})

def habit_cal(list_a):
    len_list = len(list_a)
    diff_list = [(datetime.strptime(y, '%Y-%m-%d')-datetime.strptime(x, '%Y-%m-%d')).days for x, y in zip(list_a[:-1], list_a[1:])]
    return np.mean(diff_list)
df_user["habit_user"] = df_user["created_at"].apply(lambda x: habit_cal(x))

user_habit = dict(zip(df_user.user_id, df_user.habit_user))
df_user.head()

,user_id,created_at,habit_user
0,577243,"[2019-01-03, 2019-01-13, 2019-01-19, 2019-01-2...",10.750000
1,577275,"[2019-03-08, 2019-05-11, 2019-06-09, 2019-09-1...",33.125000
2,577308,"[2019-01-27, 2019-02-09, 2019-03-02, 2019-04-0...",27.700000
3,577450,"[2019-03-22, 2019-04-26, 2019-07-08, 2019-09-13]",58.333333
4,577505,"[2019-01-03, 2019-02-01, 2019-02-16, 2019-02-2...",17.166667


In [14]:
df_item = df.groupby(['product_id'], as_index=False).agg({"created_at" : lambda x: list(sorted(set(x)))})

def purchase_cal(list_a):
    len_list = len(list_a)
    diff_list = [(datetime.strptime(y, '%Y-%m-%d')-datetime.strptime(x, '%Y-%m-%d')).days for x, y in zip(list_a[:-1], list_a[1:])]
    return np.mean(diff_list)
df_item["purchase_item"] = df_item["created_at"].apply(lambda x: purchase_cal(x))

item_habit = dict(zip(df_item.product_id, df_item.purchase_item))
df_item.head()

,product_id,created_at,purchase_item
0,113646,"[2019-01-05, 2019-02-01, 2019-02-05, 2019-02-1...",11.533333
1,113858,"[2019-03-16, 2019-03-28, 2019-04-07, 2019-04-15]",10.000000
2,114044,"[2019-03-13, 2019-04-04, 2019-04-25, 2019-06-0...",43.750000
3,115919,"[2019-01-04, 2019-01-11, 2019-01-15, 2019-01-2...",5.600000
4,115932,"[2019-03-01, 2019-03-09, 2019-04-26, 2019-05-2...",16.428571


In [3]:
df2 = df.groupby(['user_id', 'product_id'], as_index=False).agg({"created_at" : lambda x: sorted(list(x))})


In [4]:
df2.head()

,user_id,product_id,created_at
0,577243,724112,"[2019-03-13, 2019-03-25, 2019-04-22, 2019-06-1..."
1,577243,875550,"[2019-01-19, 2019-02-13, 2019-04-13, 2019-06-16]"
2,577243,895941,"[2019-01-19, 2019-01-26, 2019-01-31, 2019-02-1..."
3,577243,895955,"[2019-02-02, 2019-05-12, 2019-10-17, 2019-10-29]"
4,577243,896036,"[2019-01-03, 2019-01-13, 2019-01-19, 2019-01-2..."


In [5]:
df2["last_purchase"] = df2["created_at"].apply(lambda x: x[-1])
df2.head()

,user_id,product_id,created_at,last_purchase
0,577243,724112,"[2019-03-13, 2019-03-25, 2019-04-22, 2019-06-1...",2019-06-29
1,577243,875550,"[2019-01-19, 2019-02-13, 2019-04-13, 2019-06-16]",2019-06-16
2,577243,895941,"[2019-01-19, 2019-01-26, 2019-01-31, 2019-02-1...",2019-12-13
3,577243,895955,"[2019-02-02, 2019-05-12, 2019-10-17, 2019-10-29]",2019-10-29
4,577243,896036,"[2019-01-03, 2019-01-13, 2019-01-19, 2019-01-2...",2019-10-10


In [ ]:
df_a = pd.read_csv("answer.csv")
df_a.head()

In [ ]:
mean([1,2,3])

In [ ]:
!pip install statistics

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv

In [ ]:
dataframe = pd.read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [46]:
def myround(x, base=5):
    return base * round(x/base)

def predict_last(user_id, product_id, list_a):
    
    
    #list_a = list(set(list_a))
    len_list = len(list_a)
    diff_list = [(datetime.strptime(y, '%Y-%m-%d')-datetime.strptime(x, '%Y-%m-%d')).days for x, y in zip(list_a[:-1], list_a[1:])] 
    #sum_dates_diff = sum([(datetime.strptime(y, '%Y-%m-%d')-datetime.strptime(x, '%Y-%m-%d')).days for x, y in zip(list_a[:-1], list_a[1:])] )
    #return str(datetime.strftime(datetime.strptime(list_a[-1], '%Y-%m-%d') + timedelta(days=round(sum_dates_diff/len_list)), '%Y-%m-%d'))
    data = {"diff": diff_list}
    df_train = pd.DataFrame(data)
    
    diff_list = [myround(x, base = 3) for x in diff_list]

    
    alpha = 1
    sum_we = 0
    we = 0
    for x in diff_list[-1::-1]:
        sum_we += x*alpha
        we += alpha
        alpha *= 0.9
    
    pred4 = np.average([diff_list[-1], diff_list[-2]])
    pred1 = sum_we/we
    pred2 = user_habit[user_id]
    pred3 = item_habit[product_id]
    preds = [pred1, pred2, pred3, pred4]
    weight = [1, 0, 0, 0]
    #num_days = model_predict(df_train)
    # pick off the first group (highest frequency)
    #freqs = groupby(Counter(diff_list).most_common(), lambda x:x[1])
    #num_days = round(mean([val for val,count in next(freqs)[1]]))
    #model = ARIMA(diff_list, order=(0, 0, 1))
    #model_fit = model.fit()
    # make prediction
    #yhat = model_fit.predict(len(diff_list), len(diff_list))

    #num_days = round(sum_we/we)
    num_days = round(np.average(preds, weights = weight))
    return str(datetime.strftime(datetime.strptime(list_a[-1], '%Y-%m-%d') + timedelta(days=num_days), '%Y-%m-%d'))
  



df2["next_purchase"] = df2.apply(lambda x: predict_last(x["user_id"], x["product_id"], x["created_at"]), axis=1)

In [47]:
df2.head()

,user_id,product_id,created_at,last_purchase,next_purchase
0,577243,724112,"[2019-03-13, 2019-03-25, 2019-04-22, 2019-06-1...",2019-06-29,2019-07-26
1,577243,875550,"[2019-01-19, 2019-02-13, 2019-04-13, 2019-06-16]",2019-06-16,2019-08-05
2,577243,895941,"[2019-01-19, 2019-01-26, 2019-01-31, 2019-02-1...",2019-12-13,2020-01-22
3,577243,895955,"[2019-02-02, 2019-05-12, 2019-10-17, 2019-10-29]",2019-10-29,2020-01-24
4,577243,896036,"[2019-01-03, 2019-01-13, 2019-01-19, 2019-01-2...",2019-10-10,2019-11-10


In [48]:

df_b = pd.DataFrame( df2["next_purchase"])

df_b.to_csv("answer.csv",index = False)